In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 6+ `guest_count`
Let's look at a few tables:
- `hockey_3_text_processed.csv`
- `silvester_3_text_processed.csv`
- `valentine_3_text_processed.csv`

to get an idea of the different occasions.

###### Our initial set of **occasions** for the category 6+ bin is: 
1. `Drinking` - purpose to drink
    - `Social drinking`
    - `Social Gathering`
    - `Party`
2. `Meal` - purpose is to eat
    - `lunch meal`
    - `after work meal`
    - `Family Occasion ` - might need to further define these categories
    


### Something to think about:
- `Lunch` vs `dinner`
- `during sport events`
- `what is their average dwell time at a table`

----------

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

In [5]:
hockey_path = '../data/hockey_3_text_processed.csv'
silvester_path = '../data/silvester_3_text_processed.csv'
valentine_path = '../data/valentine_3_text_processed.csv'

In [6]:
hockey = pd.read_csv(hockey_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [7]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [8]:
fix_times(hockey)
fix_times(valentine)
fix_times(silvester)

Lets only look at interesting features

In [9]:
features = ['order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'title', 'category_id', 
       'dwell_time', 'meal_step',
       'meal_flow_last_to_close', 'total_flow_steps', 'meal_flow_step', 
       'beer_volume', 'order_day_of_week',
       'is_weekend', 'guest_count',
       'period_of_day', 
       'sharable', 'kids_meal', 'birthday',
       'total_sales_before_tax', 'item_qty', 'sales_before_tax']

In [10]:
hockey = hockey[features]
silvester = silvester[features]
valentine = valentine[features]

In [11]:
hockey.head()

,order_id,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,0,0,0,19.84,1.0,2.97
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,1.247778,1,15,18,3,0.000,Tuesday,False,1,lunch,1,0,0,19.84,1.0,16.87
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:18:25,TRAD.GUAC,2.0,1.181944,2,13,17,4,0.000,Tuesday,False,1,lunch,1,0,0,26.61,1.0,8.99
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,1.181944,1,13,17,3,0.000,Tuesday,False,1,lunch,0,0,0,26.61,1.0,17.62
4,520253565,2019-04-16 19:35:00,2019-04-16 22:39:18,2019-04-16 20:56:13,CANADIAN.Pt,1.0,3.071667,2,12,44,19,0.473,Tuesday,False,0,dinner,0,0,0,30.48,1.0,7.62


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [12]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [13]:
hockey_tables = create_tables(hockey)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

In [14]:
print("We have ", len(hockey_tables), " tables in the Hockey data")

We have  118664  tables in the Hockey data


In [15]:
print("We have ", len(valentine_tables), " tables in the Valentine data")

We have  40787  tables in the Valentine data


In [16]:
print("We have ", len(silvester_tables), " tables in the Silvester data")

We have  92107  tables in the Silvester data


In [17]:
hockey.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

# The Annotations

Pick 10 tables out of each dataframe.

In [18]:
COUNT = 10

In [19]:
import random

In [20]:
def get_tables_gc_6(tables):
    return tables[tables['guest_count'] >5]

## Annotating Hockey

Pick 10 tables of a total of 1 guests of **Hockey**.

In [21]:
hockey_6gc_tables = get_tables_gc_6(hockey_tables)
print("We have {} tables of a total of 6+ guests".format(len(hockey_6gc_tables)))

We have 2377 tables of a total of 6+ guests


In [22]:
picked_hockey_tables = random.sample(list(hockey_6gc_tables.index), COUNT)

In [24]:
picked_hockey_tables = [514594544,
 520144566,
 512856753,
 523552965,
 522777307,
 520932269,
 520961529,
 522822209,
 524870243,
 514742701]

# Table 1

In [23]:
#514534188
hockey[hockey.index == picked_hockey_tables[0]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Tex Steak Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,15.762830
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,P.B Deli Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,12.805451
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Onion Soup,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,5.904900
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fish & Chips,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,13.791244
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fount Pop,4.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,2.947521
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Fish & Chips,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,13.791244
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Tex Steak Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,15.762830
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Hot Beef Sand,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,0,0,0,199.603372,1.0,14.777037
514594544,2019-04-13 19:58:39,2019-04-13 21:23:25,2019-04-13 20:04:17,Chicken Fingers,2.0,1.412778,1,6,19,1,0.000,Saturday,True,10,dinner,1,0,0,199.603372,1.0,19.686287


# **April 13th (Saturday), 19:58 - 21:23, a total of 1.5 hours**

**Meal Steps Description:**
1. Start
  - Meal very heavy, all ordered at once

2. Coffe 30 min later
  - Dessert

      
**`Dinner`** 


**Explanation**: Because of the time and the size of the meal.

**Features that helped**: the amount of foold on the table is approxemtly the GC, `price of meal` (10 $), and `time of meal` (21:00pm). 

# Table 2

In [27]:
hockey[hockey.index == picked_hockey_tables[1]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,-0.000,Tuesday,False,22,dinner,0,0,0,3253.84,-1.0,0.000000
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,22.0,2259.481435
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 19:51:43,Deluxe Dinner,2.0,0.773333,1,0,10,0,-0.000,Tuesday,False,22,dinner,0,0,0,3253.84,-1.0,0.000000
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Pop,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,2.0,5.835438
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Evian,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,9.0,91.908142
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Guinness,1.0,0.773333,2,0,10,2,0.440,Tuesday,False,22,dinner,0,0,0,3253.84,1.0,9.336700
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Heineken,1.0,0.773333,2,0,10,2,0.660,Tuesday,False,22,dinner,0,0,0,3253.84,2.0,18.673400
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,Castle Cab Sauv,6.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,6.0,406.146456
520144566,2019-04-16 19:51:09,2019-04-16 20:37:33,2019-04-16 20:00:32,San Pellegrino,4.0,0.773333,2,0,10,2,0.000,Tuesday,False,22,dinner,0,0,0,3253.84,7.0,71.484110


**April 16th (Tuesday), 19:51 - 20:37, total 0.77 hours, 2 meal steps**

** 22 people eat dinner and some order more soft drinks and liquer 
I dont use the the dwell time here
**Meal Steps Description:**
it actually has a single meal flow and not 2 as mentioned. 
       
**Total drinks**: 5 alcholic drinks 3 soft drinks

Dinner event

**Explanation**: This is a party where the foucus is on the food, room was set for the group. 

**Features**: number of dinners which was served for the table, amount of food over the amount of drinks

rule: if the guest count is greather than 12 then it was set before, therefore it a big event and with high ratio of food over drinks it is a dinner event. 

# Table 3

In [28]:
#521093892
hockey[hockey.index == picked_hockey_tables[2]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 20:35:00,LG COORS LIGHT,1.0,4.835278,1,8,70,0,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 20:41:00,CHICKEN STIR FRY,2.0,4.835278,2,8,70,1,0.000,Thursday,False,11,dinner,0,0,0,76.75,1.0,16.00
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 21:21:00,LG COORS LIGHT,1.0,4.835278,2,8,70,11,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 22:31:00,LG COORS LIGHT,1.0,4.835278,2,8,70,28,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-11 23:27:00,LG COORS LIGHT,1.0,4.835278,2,8,70,42,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:28:00,LG COORS LIGHT,1.0,4.835278,2,8,70,57,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:37:00,LG COORS LIGHT,1.0,4.835278,2,8,70,59,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:37:00,LG COORS LIGHT,1.0,4.835278,2,8,70,59,0.591,Thursday,False,11,dinner,0,0,0,76.75,1.0,7.25
512856753,2019-04-11 20:35:00,2019-04-12 01:25:07,2019-04-12 00:52:00,PEPPERONI SML,2.0,4.835278,2,8,70,62,0.000,Thursday,False,11,dinner,0,0,0,76.75,1.0,10.00


**Social Gathering**

**April 11th (Thursday), 20:35 - 01:25, total of 4.83 hours**

**Meal Steps Description:**
1. food and beer
2. 40 minutes later: beer
3. 70 minutes later: beer
4. 60 min later: beer
5. 60 min later: 2 beer
6. 15 min later: pizza

**Total drinks: 2 cocktails, 1 mixed drink, 2 shots**





**Explanation**: Its hard to say how many people are on the table but it is less than 11 and probabily less than 6 so I classifity this as a wornk label of GC

**Features**: only single row per meal step for repeated amount of times 

# Table 4

In [29]:
hockey[hockey.index == picked_hockey_tables[3]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:42:00,SODA,4.0,1.830556,1,7,26,0,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,3.00
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:42:00,COKE,4.0,1.830556,1,7,26,0,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,3.00
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:50:00,CALAMARI,2.0,1.830556,2,7,26,2,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,13.67
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 18:50:00,CALAMARI,2.0,1.830556,2,7,26,2,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,13.67
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,CHICKN FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,21.88
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,HALIBUT&CHIPS,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,24.88
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,STEAK FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,22.43
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,STEAK FAJITA,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,0,0,0,188.77,1.0,22.43
523552965,2019-04-20 18:42:00,2019-04-20 20:31:50,2019-04-20 19:04:00,2LB WINGS,2.0,1.830556,2,7,26,5,0.00,Saturday,True,6,dinner,1,0,0,188.77,1.0,25.92


** Dinner **
Family dinner 


There is also a "kid" meal in here.

many foods and single beer with two soft drinks

**Features**: GC size of a family, rator of food to alcohol beverage is high. small number of soft drinks

# Table 5

In [30]:
hockey[hockey.index == picked_hockey_tables[4]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,JACK DANIELS DBL,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,10.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CUERVO GOLD,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,6.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CORONA,1.0,0.662778,1,2,8,0,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,CORONA,1.0,0.662778,1,2,8,0,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,RED BULL,4.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,4.25
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:06:32,HENNESSY DBL,3.0,0.662778,1,2,8,0,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,26.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,BURT REYNOLDS,3.0,0.662778,2,2,8,4,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,4.95
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,CORONA,1.0,0.662778,2,2,8,4,0.355,Saturday,True,6,night,0,0,0,132.45,1.0,7.50
522777307,2019-04-20 00:06:00,2019-04-20 00:45:46,2019-04-20 00:24:55,HENNESSY DBL,3.0,0.662778,2,2,8,4,0.000,Saturday,True,6,night,0,0,0,132.45,1.0,26.50


**April 20th (Sauturday), 00:06 - 00:45, total of 0.66 hours**

Social drinking

only drinks on the table, 2 meal step and with six rows per meal step. 


**Explanation**: Not much to explain, a group of people are drinking.


# Table 6

In [31]:
hockey[hockey.index == picked_hockey_tables[5]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:20:51,POP.,4.0,1.391944,1,8,16,0,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:20:51,CREEMORE.Pt,1.0,1.391944,1,8,16,0,0.473,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,8.72
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:28:38,POP.,4.0,1.391944,2,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:28:38,POP.,4.0,1.391944,2,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,3.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:29:39,Loaded.Mash,2.0,1.391944,1,8,16,1,0.000,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,0.99
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:30:17,SAM.ADAM.SEAS.Pt,1.0,1.391944,1,8,16,1,0.597,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,9.33
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:30:17,BELGIAN.MOON..Pt,1.0,1.391944,1,8,16,1,0.473,Wednesday,False,6,afternoon,0,0,0,111.59,1.0,8.43
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:33:39,TUNA.GUAC,2.0,1.391944,1,8,16,1,0.000,Wednesday,False,6,afternoon,1,0,0,111.59,1.0,14.97
520932269,2019-04-17 17:20:00,2019-04-17 18:43:31,2019-04-17 17:41:33,FINGERS.SAUCED,2.0,1.391944,2,8,16,2,0.000,Wednesday,False,6,afternoon,1,0,0,111.59,1.0,16.87


**April 17th (Wednesday), 17:20 - 18:43, total of 1.39 hours,.**

**Meal Steps Description:**

1. Start:
   - Drink amd beer

2. ~20 minutes after
   - Meal
       - many types of food 


**Explanation**: **IM HERE**
**Features**:  



# Table 7

In [29]:
hockey[hockey.index == picked_hockey_tables[6]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,RICKARD.RED.Pt,1.0,1.639167,1,8,21,0,0.473,Wednesday,False,10,dinner,0,0,0,214.57,1.0,8.43
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,HEINEKEN.Pt,1.0,1.639167,1,8,21,0,0.562,Wednesday,False,10,dinner,0,0,0,214.57,1.0,9.33
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:01,POP.,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,3.99
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:27,KIDBEV,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:22:27,WATER.,4.0,1.639167,1,8,21,0,0.000,Wednesday,False,10,dinner,0,0,0,214.57,1.0,0.00
520961529,2019-04-17 18:22:00,2019-04-17 20:00:21,2019-04-17 18:27:34,RICKARD.RED.Pt,1.0,1.639167,2,8,21,1,0.473,Wednesday,False,10,dinner,0,0,0,214.57,1.0,8.43


**April 11th (Thursday), 23:40 - 01:01, total of 1.35 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Guinnes, half a liter
2. 1hr later
    - drink
        - Guiness, half a liter
3. 20 minutes later
    - meal
        - chicken fingers, and sweet potato fries
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

`late casual drink & meal`

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

# Table 8

In [30]:
hockey[hockey.index == picked_hockey_tables[7]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:48:00,TITO VODKA DBL,3.0,0.503056,1,3,5,0,0.000,Saturday,True,7,night,0,0,0,951.0,12.0,150.00
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 01:59:00,GOOSE IPA TAP,1.0,0.503056,2,3,5,2,0.591,Saturday,True,7,night,0,0,0,951.0,1.0,7.25
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,1800 SILVER,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,7.75
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,1800 SILVER,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,7.75
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,1800 SILVER,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,7.75
522822209,2019-04-20 01:48:00,2019-04-20 02:18:11,2019-04-20 02:02:00,TITO VODKA DBL,3.0,0.503056,1,3,5,2,0.000,Saturday,True,7,night,0,0,0,951.0,1.0,12.50


**April 23rd (Tuesday), 17:44 - 19:25, total of 1.7 hours, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Keiths, half a liter
2. 45 min later
    - drink
        - Keiths, half a liter
3. 20 minutes later
    - meal
        - chicken mac and cheese (bar food.. amarican bar food. I miss mac&cheese)
        
**total drinks**: 2 beers, **total meals**: 1 "sharable/late night" food.

**`casual drink & meal`**

`late_night`, `beer`, `dwell_time`, `sharable_food`(bar food).

**Explanation**: Its a classic table, maybe also watchign a game or something. Starts with a beer and sits for an hour till person orders another drink. Then twenty minutes later orders food and closes the table.
Its a late night, the person is drinking beer, and eating bar type/sharable food. Similar to previous table.

**Features**: staying at the table for 2 hours, and having a long time between drinks hints at a casual drink/game watching table.

# Valentine

Pick 5 tables of total of 1 guests for **Valentine**.

In [31]:
valentine_gc = get_tables_gc_6(valentine_tables)

print("We have {} tables of a total of 1 guests".format(len(valentine_1gc)))

NameError: name 'valentine_1gc' is not defined

In [33]:
picked_valentine_tables = random.sample(list(valentine_gc.index), COUNT)

In [34]:
picked_valentine_tables

[447987028,
 447891027,
 435047946,
 435109182,
 447371863,
 434735609,
 447098446,
 447066407,
 435142865,
 447168302]

## Table 1

In [35]:
valentine[valentine.index == picked_valentine_tables[0]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:05:00,KIDBEV,4.0,1.011111,1,1,14,0,0.000,Friday,True,6,dinner,0,1,0,105.19,1.0,0.00
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:05:00,KIDPOP,4.0,1.011111,1,1,14,0,0.000,Friday,True,6,dinner,0,1,0,105.19,1.0,0.00
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:05:00,POP.,4.0,1.011111,1,1,14,0,0.000,Friday,True,6,dinner,0,0,0,105.19,1.0,3.99
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:05:00,GOOSE.IPA.XL,1.0,1.011111,1,1,14,0,0.591,Friday,True,6,dinner,0,0,0,105.19,1.0,10.30
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:05:00,KIDPOP,4.0,1.011111,1,1,14,0,0.000,Friday,True,6,dinner,0,1,0,105.19,1.0,0.00
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:05:00,GOOSE.IPA.XL,1.0,1.011111,1,1,14,0,0.591,Friday,True,6,dinner,0,0,0,105.19,1.0,10.30
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:17:00,K.STEAK,2.0,1.011111,2,1,14,3,0.000,Friday,True,6,dinner,0,0,0,105.19,1.0,11.25
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:17:00,WINGS,2.0,1.011111,2,1,14,3,0.000,Friday,True,6,dinner,1,0,0,105.19,1.0,0.00
447987028,2019-02-15 18:05:00,2019-02-15 19:05:40,2019-02-15 18:17:00,$Gluten.Free.Bun$,2.0,1.011111,2,1,14,3,0.000,Friday,True,6,dinner,0,0,0,105.19,1.0,1.00


**Feb 10th (Sunday), 14:13 - 17:18, total of ~3 hour, 5 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - martini
        - Beefeater ??? -cheap, maybe shot?
        - water
2. 15 min later
    - meal
        - pad thai chicken
    - drink
        - beefeater ???
3. 40 min later
    - drink
        - "bad manager" beer, half liter
      
4. 35 min later
    - drink
        - red pt, half liter
5. 1 hour later
    - drink
        - red pt, hald liter
        

**total drinks**: 4 or 6, depending if we cound beefeater, **total meals**: 1 meal.

**`afternoon drink & meal`**

`dwell_time`, `amount of alcohol`, `time between drinks`

**Explanation**: This one was not easy. 3 whole hours at a bar. at first it looks like a mistake, but then you see that there are actual steps, where you start with a drink, eat a meal, and finish with two more drinks. Hard to say what type of occasion this is, but I would say a afternoon meal and drink (not casual?), since there is a meal consumed, and a good amount of drink and beer consumed.

- I wanted to say casual at first, but maybe because of the martini and amount of total alcohol its not casual?.. but its 2pm :/


# Table 2

In [36]:
valentine[valentine.index == picked_valentine_tables[1]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
447891027,2019-02-15 12:33:00,2019-02-15 13:29:01,2019-02-15 12:33:00,LG CANADIAN,1.0,0.933611,1,4,12,0,0.591,Friday,True,6,lunch,0,0,0,34.75,1.0,7.25
447891027,2019-02-15 12:33:00,2019-02-15 13:29:01,2019-02-15 12:35:00,REUBEN,2.0,0.933611,1,4,12,0,0.000,Friday,True,6,lunch,0,0,0,34.75,1.0,13.00
447891027,2019-02-15 12:33:00,2019-02-15 13:29:01,2019-02-15 12:50:00,LG CANADIAN,1.0,0.933611,2,4,12,3,0.591,Friday,True,6,lunch,0,0,0,34.75,1.0,7.25
447891027,2019-02-15 12:33:00,2019-02-15 13:29:01,2019-02-15 13:10:00,LG CANADIAN,1.0,0.933611,3,4,12,8,0.591,Friday,True,6,lunch,0,0,0,34.75,1.0,7.25


**Feb 10th (Sunday), 21:56 - 22:25, total of 0.5 hours (30 minutes), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - chicken fingers
        
**total meals**: 1 late night sharable meal

**`late night munch`** - a sharble item late at night

`sharable`, `late_night`, `1_step/1 item`


**Explanation**: Looks like a late munch. I would even say maybe its a take out, then again it could be a person stopped for wings and then left.



# Table 3

In [37]:
valentine[valentine.index == picked_valentine_tables[2]].sort_values(by=['order_item_time'])

,order_time,order_time_closed,order_item_time,title,category_id,dwell_time,meal_step,meal_flow_last_to_close,total_flow_steps,meal_flow_step,beer_volume,order_day_of_week,is_weekend,guest_count,period_of_day,sharable,kids_meal,birthday,total_sales_before_tax,item_qty,sales_before_tax
order_id,,,,,,,,,,,,,,,,,,,,,
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:15:00,BUDWEISER,1.0,1.080278,1,1,14,0,0.341,Monday,False,17,afternoon,0,0,0,94.2,1.0,5.54
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:39:00,BUDWEISER,1.0,1.080278,2,1,14,6,0.341,Monday,False,17,afternoon,0,0,0,94.2,1.0,5.54
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:39:00,BUD LIGHT,1.0,1.080278,2,1,14,6,0.341,Monday,False,17,afternoon,0,0,0,94.2,1.0,5.54
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:40:00,BELVEDERE,3.0,1.080278,2,1,14,6,0.000,Monday,False,17,afternoon,0,0,0,94.2,1.0,8.50
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:40:00,BELVEDERE,3.0,1.080278,2,1,14,6,0.000,Monday,False,17,afternoon,0,0,0,94.2,1.0,8.50
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:40:00,PERRIER,4.0,1.080278,2,1,14,6,0.000,Monday,False,17,afternoon,0,0,0,94.2,1.0,3.50
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:40:00,PERRIER,4.0,1.080278,2,1,14,6,0.000,Monday,False,17,afternoon,0,0,0,94.2,1.0,3.50
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:48:00,CHICKEN BITES,2.0,1.080278,3,1,14,8,0.000,Monday,False,17,afternoon,0,0,0,94.2,1.0,7.00
435047946,2019-02-11 16:15:00,2019-02-11 17:19:49,2019-02-11 16:51:00,BELVEDERE,3.0,1.080278,3,1,14,8,0.000,Monday,False,17,afternoon,0,0,0,94.2,1.0,8.50


**Feb 13th (Wednesday), 13:08 - 14:07, total of ~1 hour, 3 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - soda pop
2. 4 min later
    - meal
        - chicken fingers
3. 35 minutes after
    - dessert
        - brownie

**total drinks**: 1 non alc drink, **total meals**: 1 lunch meal, and 1 dessert.

**`lunch`**

`lunch_time`, `meal`, `dwell_time`

**Explanation**: classic lunch, even went for dessert. Lunches shouldn't usually dwell too long. no alcohol and a single meal.

**Features**: time of meal (1pm), no alcohol, and spent about an hour.

# Table 4

In [ ]:
valentine[valentine.index == picked_valentine_tables[3]].sort_values(by=['order_item_time'])

**Feb 10th (Tuesday), 00:50 - 02:07, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - amsterdam beer, half liter
2. 1 hr and 10 min later
    - drink
        - mills, half liter

**total drinks**: 2 beer drinks

**`late casual drink`** I think theres a difference between drinking late at night beer, and hard liquer.

    `time_between_drinks`, `beer`, `late_night` 

**Explanation**: pretty straight forward, late at night, drinking beer and taking its time. drinking a beer over an hour and then a second one.

**Features**: an hour between beers, having a total of two beers (seems like a common number of beers), and late night

#### 5. Table 446744761

In [ ]:
valentine[valentine.index == picked_valentine_tables[4]].sort_values(by=['order_item_time'])

**Feb 12th (Tuesday), 18:37 - 19:57, total of 1.3 hours, 2 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - Wine
2. 10 min later
    - meal
        - Bacon Filet (huh?.. fancy)

**total drinks**: 1 drink, **total meals**: 1 dinner meal.

**`dinner`** (maybe even a celebratory dinner, or "pinuk" dinner? "treat yo self" type dinner.

`wine`, `expensive_meal`, `dinner_time`

**Explanation**: classic dinner table, although important to note that the wine and filet makes it a bit more fancy, special occasion type:)

**Features**: expensive meal -(30$), dinner time (6pm), `wine`.

--------------

# Silvester

In [ ]:
silvester_gc = get_tables_gc_6(silvester_tables)
print("We have {} tables of a total of 1 guests".format(len(silvester_gc)))

In [ ]:
picked_silvester_tables = random.sample(list(silvester_gc.index), COUNT)

In [ ]:
picked_silvester_tables

# Table 1

In [ ]:
silvester[silvester.index == picked_silvester_tables[0]].sort_values(by=['order_item_time'])

**Jan 03rd (Thursday), 17:39 - 00:01, total of 6 hours (mistake), 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - veggie burger


**total meals**: 1 dinner meal.

**`dinner`** 

`dinner_time`, `1_meal`

**Explanation**: classic dinner for 1. Only mistake I see is they didnt close the table, and so the dwell time says 6 hours.


# Table 2

In [ ]:
silvester[silvester.index == picked_silvester_tables[1]].sort_values(by=['order_item_time'])

**Dec 30th (Sunday), 22:47 - 23:45, 0.9 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - non alcoholic, sassafraz??
        - guinness, half liter
    - meal
        - Drunken platter, 29\$
        - loaded mash, 0.9$


**Total drinks**: 1 half liter, **total meals**: 1 large meal

**`Late dinner`** 

`late night`, `1 drink`, `1 meals`, `1 expensive meal`

**Explanation**: Another classic dinner for 1, but late at night.


# Table 3

In [ ]:
silvester[silvester.index == picked_silvester_tables[2]].sort_values(by=['order_item_time'])

**Dec 28th (Friday), 22:17 - 23:13, 0.9 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - drink
        - cocktail, strawberry paloma
    - meal
        - chicken wrap


**Total drinks**: 1 cocktail, **total meals**: 1 meal

**`Late dinner`** 

`late night`, `1 drink`, `1 meal`, 

**Explanation**: Another classic dinner for 1, but late at night.


# Table 4

In [ ]:
silvester[silvester.index == picked_silvester_tables[3]].sort_values(by=['order_item_time'])

**Dec 26th (Wednesday), 17:42 - 18:15, 0.55 hrs, 1 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - fajitas chicken shrimp (big meal)


**total meals**: 1 meal

**`dinner`** 

`dinner time`, `1 meal`,`big meal` 

**Explanation**: classic dinner for 1, a lot of food.


# Table 5

In [ ]:
silvester[silvester.index == picked_silvester_tables[4]].sort_values(by=['order_item_time'])

**Dec 25th (Tuesday), 18:18 - 22:18, 4 hrs, 6 meal steps.**

**Meal Steps Description:**
1. Start:
    - meal
        - potatos (chips?)
    - drink
        - keiths tap, half liter
2. 25 minutes later
    - drink
        - keiths tap, half liter
3. 20 minutes later
    - meal
        - perogies, snack
4. 50 min later
    - drink
        - keiths tap, half liter
5. 25 minutes later
    - drink
        - keiths tap, half liter
6. 30 minutes later
    - drink
        - keiths tap, half liter


**total meals**: 2 snack type foods, **total drinks**: 5 beers, half liters

**`Drinking`** - Dinner time 

`dinner time`, `large amount of alcohol`, `dwell time`, `number of steps` 

**Explanation**: Christams day drinking:(, right at dinner time, eats two somewhat smaller type foods, I wouldnt say they were "meals", more drinking food. The person drinks 5 half liter beers, the first two 25 min apart, and the last three also about 25 min apart.
